# LangChain Cohere Agent using Tools

## Install Libraries

In [ ]:
# ! pip install streamlit
# ! pip install qdrant-client
# ! pip install langchain
# ! pip install langchain-community
# ! pip install langchain-qdrant
# ! pip install langchain-cohere
# ! pip install tiktoken
# ! pip install cohere
# ! pip install bs4
# ! pip install markdown
# ! pip install wikipedia
# ! pip install pyowm
# ! pip install yfinance

## Imports for Tool Use and Keys
### API Keys
* https://cohere.com/
* https://www.alphavantage.co/
* https://qdrant.tech/
* https://openweathermap.org/
* https://tavily.com/
* https://polygon.io/


In [1]:
from IPython.display import HTML
import markdown
import warnings
import streamlit as st
import os
from langchain_cohere.llms import Cohere
from langchain_core.messages import HumanMessage
from langchain_core.prompts import ChatPromptTemplate
from langchain_cohere import ChatCohere, create_cohere_react_agent
from langchain.agents import AgentExecutor
from cohere_tools import get_internet_search_tool, get_wikipedia_tool, get_openweathermap_tool
from yfinance_tools import get_business_summary_tool, get_ticker_history_tool, get_ticker_info_tool
from alpha_vantage_tools import (
    get_search_symbol_tool,
    get_exchange_rate_tool,
    # get_time_series_daily_tool,
    # get_time_series_weekly_tool,
    # get_market_news_sentimen_tool,
    # get_top_gainers_losers_tool,
)

warnings.filterwarnings("ignore")

ALPHAVANTAGE_API_KEY = st.secrets["ALPHAVANTAGE_API_KEY"]
os.environ["ALPHAVANTAGE_API_KEY"] = st.secrets["ALPHAVANTAGE_API_KEY"]

COHERE_API_KEY = st.secrets["COHERE_API_KEY"]
os.environ["COHERE_API_KEY"] = st.secrets["COHERE_API_KEY"]

QDRANT_HOST = st.secrets["QDRANT_HOST"]
QDRANT_API_KEY = st.secrets["QDRANT_API_KEY"]

OPEN_WEATHER_API_KEY = st.secrets["OPEN_WEATHER_API_KEY"]
os.environ["OPENWEATHERMAP_API_KEY"] = OPEN_WEATHER_API_KEY

POLYGON_API_KEY = st.secrets["POLYGON_API_KEY"]
os.environ["POLYGON_API_KEY"] = st.secrets["POLYGON_API_KEY"]

TAVILY_API_KEY = st.secrets["TAVILY_API_KEY"]
os.environ["TAVILY_API_KEY"] = st.secrets["TAVILY_API_KEY"]

## Invoke ChatCohere

In [8]:
chat = ChatCohere()
messages = [HumanMessage(content="knock knock")]
print(chat.invoke(messages))

content="Who's there?" additional_kwargs={'documents': None, 'citations': None, 'search_results': None, 'search_queries': None, 'is_search_required': None, 'generation_id': '8e78cb0d-af71-425c-aec0-73f4e2ac175f', 'token_count': {'input_tokens': 68, 'output_tokens': 4}} response_metadata={'documents': None, 'citations': None, 'search_results': None, 'search_queries': None, 'is_search_required': None, 'generation_id': '8e78cb0d-af71-425c-aec0-73f4e2ac175f', 'token_count': {'input_tokens': 68, 'output_tokens': 4}} id='run-5561524f-54ce-4c9d-bc1e-c55958d9d77a-0'


## Invoke ChatCohere using ChatPromptTemplate

In [9]:
prompt = ChatPromptTemplate.from_template("Tell me a joke about {topic}")
chain = prompt | chat
chain.invoke({"topic": "bears"})

AIMessage(content='Why don’t bears wear socks?\n\nThey have bear feet!', additional_kwargs={'documents': None, 'citations': None, 'search_results': None, 'search_queries': None, 'is_search_required': None, 'generation_id': 'c9c7e2b2-8d70-4a99-814f-c36f593e3de5', 'token_count': {'input_tokens': 72, 'output_tokens': 15}}, response_metadata={'documents': None, 'citations': None, 'search_results': None, 'search_queries': None, 'is_search_required': None, 'generation_id': 'c9c7e2b2-8d70-4a99-814f-c36f593e3de5', 'token_count': {'input_tokens': 72, 'output_tokens': 15}}, id='run-87c97268-c707-438f-8c86-ccc31919c308-0')

## Invoke Cohere

In [12]:
llm = Cohere()
display(HTML(markdown.markdown(llm.invoke("Summarize the theory of Kahnemann and Tversky"))))

## Tavily Internet Search Tool

In [44]:
llm = ChatCohere()
internet_search = get_internet_search_tool()
prompt = ChatPromptTemplate.from_template("In which year had {stock_name} its IPO? Tell me the year and the date.")
agent = create_cohere_react_agent(llm, [internet_search], prompt)
agent_executor = AgentExecutor(agent=agent, tools=[internet_search], verbose=True)
response = agent_executor.invoke(
    {
        "stock_name": "American Superconductor",
    }
)

display(HTML(markdown.markdown(response['output'])))



> Entering new AgentExecutor chain...

I will search for 'American Superconductor IPO date' and then write an answer.
{'tool_name': 'internet_search', 'parameters': {'query': 'American Superconductor IPO date'}}
[{'url': 'https://stockanalysis.com/stocks/amsc/', 'content': 'Get a real-time American Superconductor Corporation (AMSC) stock price quote with breaking news, financials, statistics, charts and more. ... Earnings Date: May 29, 2024: ... IPO Date Dec 11, 1991. Employees 328. Stock Exchange NASDAQ. Ticker Symbol AMSC. Full Company Profile.'}, {'url': 'https://www.nasdaq.com/press-release/amsc-announces-proposed-public-offering-of-common-stock-2024-01-30', 'content': 'AYER, Mass., Jan. 30, 2024 (GLOBE NEWSWIRE) -- American Superconductor Corporation (Nasdaq: AMSC), a leading system provider of megawatt-scale power resiliency solutions that orchestrate the ...'}, {'url': 'https://finance.yahoo.com/news/amsc-announces-proposed-public-offering-210100534.html', 'content': 'AYER, Ma

## Wikipedia Tool

In [45]:
llm = ChatCohere()
prompt = ChatPromptTemplate.from_template("{input}")
wikipedia_tool = get_wikipedia_tool()
agent = create_cohere_react_agent(llm, [wikipedia_tool], prompt)
agent_executor = AgentExecutor(agent=agent, tools=[wikipedia_tool], verbose=True)
response = agent_executor.invoke(
    {
        "input": "Explain the theory of Kahnemann and Tversky",
    }
)

display(HTML(markdown.markdown(response["output"])))



> Entering new AgentExecutor chain...

I will search for 'Kahnemann and Tversky theory' and write an answer based on the results.
{'tool_name': 'wikipedia-tool', 'parameters': {'query': 'Kahnemann and Tversky theory'}}
Page: Daniel Kahneman
Summary: Daniel Kahneman (; Hebrew: דניאל כהנמן; March 5, 1934 – March 27, 2024) was an Israeli-American cognitive scientist best-known for his work on the psychology of judgment and decision-making. He is also known for his work in behavioral economics, for which he was awarded the 2002 Nobel Memorial Prize in Economic Sciences together with Vernon L. Smith. Kahneman's published empirical findings challenge the assumption of human rationality prevailing in modern economic theory. Kahneman became known as the "grandfather of behavioral economics."
With Amos Tversky and others, Kahneman established a cognitive basis for common human errors that arise from heuristics and biases, and developed prospect theory. In 2011, Kahneman was named by Foreign P

## Run tool without agent

In [20]:
wikipedia_tool.run("David Beckham")

"Page: David Beckham\nSummary: David Robert Joseph Beckham  ( BEK-əm; born 2 May 1975) is an English former professional footballer, the president and co-owner of Inter Miami CF and co-owner of Salford City. Primarily a right winger and known for his range of passing, crossing ability and free-kick taking, Beckham has been hailed as one of the greatest and most recognisable midfielders of his generation, as well as one of the best free-kick takers of all time. He won 19 major trophies in his career, and is the first English player to win league titles in four different countries: England, Spain, the United States and France.\nBeckham's professional club career began with Manchester United, where he made his first-team debut in 1992 at age 17. With United, he won the Premier League title six times, the FA Cup twice, the FA Charity Shield twice, the Intercontinental Cup and the UEFA Champions League in 1999. He then played four seasons with Real Madrid, winning the La Liga championship i

## Openweathermap Tool

In [46]:
llm = ChatCohere()
prompt = ChatPromptTemplate.from_template("{input}")
tools = [get_openweathermap_tool(llm), get_wikipedia_tool()]
agent = create_cohere_react_agent(tools=tools, llm=llm, prompt=prompt)
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)
response = agent_executor.invoke({"input": "How is the weather in Salvador da Bahia?"})
display(HTML(markdown.markdown(response["output"])))



> Entering new AgentExecutor chain...

I will search for the weather in Salvador da Bahia and write an answer based on the information I find.
{'tool_name': 'open_weather_map', 'parameters': {'location': 'Salvador da Bahia'}}
In Salvador da Bahia, the current weather is as follows:
Detailed status: overcast clouds
Wind speed: 7.42 m/s, direction: 180°
Humidity: 79%
Temperature: 
  - Current: 28.98°C
  - High: 28.98°C
  - Low: 28.98°C
  - Feels like: 34.47°C
Rain: {}
Heat index: None
Cloud cover: 100%Relevant Documents: 0
Cited Documents: 0
Answer: In Salvador da Bahia, the weather is currently overcast with a temperature of 28.98°C. It feels like 34.47°C.
Grounded answer: In Salvador da Bahia, the weather is currently <co: 0>overcast</co: 0> with a <co: 0>temperature of 28.98°C</co: 0>. It <co: 0>feels like 34.47°C</co: 0>.

> Finished chain.


## Multiple Tool Use
1. wiki-tool
2. open_weather_map
3. internet_search

In [25]:

llm = ChatCohere()
tools = [get_openweathermap_tool(llm), get_internet_search_tool(), get_wikipedia_tool()]

prompt = ChatPromptTemplate.from_template(
    "Use the right tool to answer the following question. If wikipedia_tool delivers no result use internet_search: {input}"
)
agent = create_cohere_react_agent(tools=tools, llm=llm, prompt=prompt)
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

print(agent_executor.invoke({"input": "Who was George Washington?"}))
print(
    agent_executor.invoke(
        {"input": "What's the weather like in Mortugaba, Bahia, Brazil?"}
    )
)
print(agent_executor.invoke({"input": "Whats in the news today?"}))
print(
    agent_executor.invoke({"input": "How did the Microsoft stock perform last week?"})
)



> Entering new AgentExecutor chain...

I will use Wikipedia to answer this question.
{'tool_name': 'wikipedia-tool', 'parameters': {'query': 'Who was George Washington?'}}
Page: George Washington Carver
Summary: George Washington Carver (c. 1864 – January 5, 1943) was an American agricultural scientist and inventor who promoted alternative crops to cotton and methods to prevent soil depletion. He was one of the most prominent black scientists of the early 20th century.
While a professor at Tuskegee Institute, Carver developed techniques to improve types of soils depleted by repeated plantings of cotton. He wanted poor farmers to grow other crops, such as peanuts and sweet potatoes, as a source of their own food and to improve their quality of life. Under his leadership, the Experiment Station at Tuskegee published over forty practical bulletins for farmers, many written by him, which included recipes; many of the bulletins contained advice for poor farmers, including combating soil d

## Yfinance Tools

In [12]:
llm = ChatCohere()
prompt = ChatPromptTemplate.from_template("{input}")
tools = [get_business_summary_tool(),get_ticker_history_tool(), get_ticker_info_tool(), get_search_symbol_tool()]
agent = create_cohere_react_agent(tools=tools, llm=llm, prompt=prompt)
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

## Business Summary Tool

In [17]:
response = agent_executor.invoke({"input": "What is the stock market symbol for Microsoft? Then tell me how the company did perform the last 3mo. Calculate the return for that period."})
display(HTML(markdown.markdown(response['output'])))



> Entering new AgentExecutor chain...

First, I will search for the stock market symbol for Microsoft. Then, I will use the symbol to find out how the company performed in the last 3 months.
{'tool_name': 'av_search_symbols', 'parameters': {'tool_input': 'Microsoft'}}
{'bestMatches': [{'1. symbol': 'MSF0.FRK', '2. name': 'MICROSOFT CORP. CDR', '3. type': 'Equity', '4. region': 'Frankfurt', '5. marketOpen': '08:00', '6. marketClose': '20:00', '7. timezone': 'UTC+02', '8. currency': 'EUR', '9. matchScore': '0.6429'}, {'1. symbol': 'MSFT', '2. name': 'Microsoft Corporation', '3. type': 'Equity', '4. region': 'United States', '5. marketOpen': '09:30', '6. marketClose': '16:00', '7. timezone': 'UTC-04', '8. currency': 'USD', '9. matchScore': '0.6154'}, {'1. symbol': '0QYP.LON', '2. name': 'Microsoft Corporation', '3. type': 'Equity', '4. region': 'United Kingdom', '5. marketOpen': '08:00', '6. marketClose': '16:30', '7. timezone': 'UTC+01', '8. currency': 'USD', '9. matchScore': '0.6000'}

## Ticker History Tool

In [21]:
response = agent_executor.invoke({"input": "How did BRK-B perform the last 5d?"})

display(HTML(markdown.markdown(response['output'])))
documents = response['citations'][0].documents
content = documents[0]['output']
display(HTML(markdown.markdown(content)))



> Entering new AgentExecutor chain...

I will search for the ticker history of BRK-B for the last 5 days.
{'tool_name': 'yf_get_ticker_history', 'parameters': {'ticker_symbol': 'BRK-B', 'period': '5d'}}
<table border="1" class="dataframe">
  <thead>
    <tr style="text-align: right;">
      <th></th>
      <th>Open</th>
      <th>High</th>
      <th>Low</th>
      <th>Close</th>
      <th>Volume</th>
      <th>Dividends</th>
      <th>Stock Splits</th>
    </tr>
    <tr>
      <th>Date</th>
      <th></th>
      <th></th>
      <th></th>
      <th></th>
      <th></th>
      <th></th>
      <th></th>
    </tr>
  </thead>
  <tbody>
    <tr>
      <th>2024-05-29 00:00:00-04:00</th>
      <td>403.000000</td>
      <td>405.869995</td>
      <td>401.579987</td>
      <td>404.089996</td>
      <td>3426800</td>
      <td>0.0</td>
      <td>0.0</td>
    </tr>
    <tr>
      <th>2024-05-30 00:00:00-04:00</th>
      <td>403.769989</td>
      <td>409.040009</td>
      <td>402.769989</td>
      

,Open,High,Low,Close,Volume,Dividends,Stock Splits
Date,,,,,,,
2024-05-29 00:00:00-04:00,403.000000,405.869995,401.579987,404.089996,3426800,0.0,0.0
2024-05-30 00:00:00-04:00,403.769989,409.040009,402.769989,408.609985,2905300,0.0,0.0
2024-05-31 00:00:00-04:00,408.089996,415.399994,406.880005,414.399994,5190000,0.0,0.0
2024-06-03 00:00:00-04:00,414.890015,416.720001,408.779999,414.790009,4903000,0.0,0.0
2024-06-04 00:00:00-04:00,414.890015,415.890015,406.440094,406.869995,1762127,0.0,0.0


## Ticker Info Tool

In [29]:
llm = ChatCohere()
prompt = ChatPromptTemplate.from_template("I need detailed information about {ticker_symbol}. Produce structured output for the 25 most important informations")
tools = [get_ticker_info_tool()]
agent = create_cohere_react_agent(tools=tools, llm=llm, prompt=prompt)
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)
response = agent_executor.invoke({"ticker_symbol": "META"})
display(HTML(markdown.markdown(response['output'])))



> Entering new AgentExecutor chain...

I will use the yf_get_ticker_info tool to find the information requested.
{'tool_name': 'yf_get_ticker_info', 'parameters': {'ticker_symbol': 'META'}}
{'address1': '1 Meta Way', 'city': 'Menlo Park', 'state': 'CA', 'zip': '94025', 'country': 'United States', 'phone': '650 543 4800', 'website': 'https://investor.fb.com', 'industry': 'Internet Content & Information', 'industryKey': 'internet-content-information', 'industryDisp': 'Internet Content & Information', 'sector': 'Communication Services', 'sectorKey': 'communication-services', 'sectorDisp': 'Communication Services', 'longBusinessSummary': 'Meta Platforms, Inc. engages in the development of products that enable people to connect and share with friends and family through mobile devices, personal computers, virtual reality headsets, and wearables worldwide. It operates in two segments, Family of Apps and Reality Labs. The Family of Apps segment offers Facebook, which enables people to share,

## Alpha Vantage Tool

In [2]:
llm = ChatCohere()

prompt = ChatPromptTemplate.from_template("{input}")

tools = [
    # get_search_symbol_tool(),
    get_exchange_rate_tool(),
    # get_time_series_daily_tool(),
    # get_time_series_weekly_tool(),
    # get_market_news_sentimen_tool(),
    # get_top_gainers_losers_tool(),
]

agent = create_cohere_react_agent(tools=tools, llm=llm, prompt=prompt)

agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)
# alpha_vantage = AlphaVantageAPIWrapper()
# alpha_vantage._get_exchange_rate("USD", "JPY")
# alpha_vantage._get_time_series_daily("MSFT")
# alpha_vantage._get_time_series_weekly("IBM")
# alpha_vantage.search_symbols("American Superconductor")
# alpha_vantage._get_market_news_sentiment("Microsoft")
# alpha_vantage._get_top_gainers_losers()
# response = agent_executor.invoke(
#     {"input": "What is the symbol for American Superconductor?"}
# )
response = agent_executor.invoke({"input": "How much is one USD in EUR?"})
# response = agent_executor.invoke({"input": "Who are the gainers and losers at the stock exchange?"})
# response = agent_executor.invoke({"input": "How is Microsoft performing historically?"})
# response = agent_executor.invoke({"input": "How did Apple perform the last time?"})
# response = agent_executor.invoke({"input": "What is in the news about Nvidia?"})



> Entering new AgentExecutor chain...

I will use the currency conversion tool to find out how much one USD is in EUR.
{'tool_name': 'av_get_exchange_rate', 'parameters': {'from_currency': 'USD', 'to_currency': 'EUR'}}
{'Realtime Currency Exchange Rate': {'1. From_Currency Code': 'USD', '2. From_Currency Name': 'United States Dollar', '3. To_Currency Code': 'EUR', '4. To_Currency Name': 'Euro', '5. Exchange Rate': '0.91880000', '6. Last Refreshed': '2024-06-04 16:57:01', '7. Time Zone': 'UTC', '8. Bid Price': '0.91875000', '9. Ask Price': '0.91884000'}}


> Finished chain.


In [3]:
response['output']

{'Realtime Currency Exchange Rate': {'1. From_Currency Code': 'USD',
  '2. From_Currency Name': 'United States Dollar',
  '3. To_Currency Code': 'EUR',
  '4. To_Currency Name': 'Euro',
  '5. Exchange Rate': '0.91880000',
  '6. Last Refreshed': '2024-06-04 16:57:01',
  '7. Time Zone': 'UTC',
  '8. Bid Price': '0.91875000',
  '9. Ask Price': '0.91884000'}}

## Imports for Embeddings and Qdrant

In [36]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.document_loaders import WebBaseLoader
from langchain.embeddings.cohere import CohereEmbeddings
from langchain.vectorstores import Qdrant
from langchain_community.vectorstores import Qdrant
from langchain.prompts import PromptTemplate
from langchain.chains import ConversationalRetrievalChain
from langchain.chains.conversation.memory import ConversationSummaryMemory
import qdrant_client

## Build Qdrant Vector Store

In [ ]:
# web_links = ["https://www.fielmann-group.com/ueber-uns/?",
#              "https://de.wikipedia.org/wiki/Fielmann",
#              "https://de.wikipedia.org/wiki/G%C3%BCnther_Fielmann",
#              "https://de.wikipedia.org/wiki/Marc_Fielmann"] 

web_links = ["https://python.langchain.com/v0.2/docs/introduction",
             "https://python.langchain.com/v0.2/docs/concepts",
             "https://python.langchain.com/v0.2/docs/integrations/tools",
             "https://python.langchain.com/v0.2/docs/integrations/vectorstores",
             "https://python.langchain.com/v0.2/docs/integrations/document_loaders",
             "https://python.langchain.com/v0.2/docs/integrations/retrievers",
             "https://python.langchain.com/v0.2/docs/integrations/vectorstores/qdrant",
             "https://python.langchain.com/v0.2/docs/integrations/providers/cohere/#react-agent",
             "https://de.wikipedia.org/wiki/Herz#:~:text=Das%20Herz%20(lateinisch%20Cor%2C%20griechisch,die%20Versorgung%20aller%20Organe%20sichert."]
 

loader = WebBaseLoader(web_links)
document=loader.load()

text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=0)
texts = text_splitter.split_documents(document)

embeddings = CohereEmbeddings(model = "embed-english-v2.0")

qdrant = Qdrant.from_documents(
    document,
    embeddings,
    url=QDRANT_HOST,
    api_key=QDRANT_API_KEY,
    prefer_grpc=True,
    collection_name="rag_documents",
    force_recreate=True,
)


## Load Qdrant Vector Store

In [37]:
client = qdrant_client.QdrantClient(
    url=QDRANT_HOST,
    api_key=QDRANT_API_KEY,
)
embeddings = CohereEmbeddings(model="embed-english-v2.0")
vector_store = Qdrant(
    client = client,
    collection_name = "rag_documents",
    embeddings = embeddings
)
print("connection established !")

connection established !


## Use Qdrant Vector Store as RAG Retriever for Cohere

In [38]:
prompt_template = """
You are a bot to answer questions from a document.
    
You will be given a context of the conversation made so far followed by a question, 
give the answer to the question using the context. 
The answer should be short, straight and to the point. If you don't know the answer, reply that the answer is not available.

Context: {context}

Question: {question}
Answer:"""

PROMPT = PromptTemplate(
    template=prompt_template, input_variables=["context", "question"]
)

chain_type_kwargs = { "prompt" : PROMPT }

llm = ChatCohere()

template = (
        """Combine the chat history and follow up question into 
        a standalone question. 
        If chat hsitory is empty, use the follow up question as it is.
        Chat History: {chat_history}
        Follow up question: {question}"""
    )

prompt = PromptTemplate.from_template(template)

chain = ConversationalRetrievalChain.from_llm(     
    llm = llm,
    chain_type = "stuff",
    memory = ConversationSummaryMemory(llm = llm, memory_key='chat_history', input_key='question', output_key= 'answer', return_messages=True),
    retriever = vector_store.as_retriever(search_type="mmr"),
    condense_question_prompt = prompt,
    return_source_documents=False,
    combine_docs_chain_kwargs=chain_type_kwargs,
)
customer_prompt = "What are tools in LangChain?"


llm_response = chain.invoke(
                {
                    "context": chain.memory.buffer,
                    "question": customer_prompt,
                },
                return_only_outputs=True,
            )

display(HTML(markdown.markdown(llm_response['answer'])))
